Calculates discounted cashflows for Fixed-Floating and OIS (TRAINING DATA)

In [23]:
# 1. Imports
import os
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# Paths
#r'C:\Users\gusta\Documents\KTH\TriOptima\trioptima/'
#'/Users/elliotlindestam/Documents/Skola/Indek icloud/trioptima/'

# 2. Configuration
your_path = r'/Users/elliotlindestam/Documents/Skola/Indek icloud/trioptima/'
train_folder_path = your_path + '6.Active Data/Train Model Data/'
test_folder_path = your_path + '6.Active Data/Test Data/'

# 3. Functions
def convert_to_usd(row,exchange_rates):
    return row['leg1NotionalAmount'] * exchange_rates.get(row['leg1NotionalCurrency'], 1)

def calculate_cashflows(row, period, freq):
    cashflows = []
    start_date = row['effectiveDate']
    end_date = row['expirationDate']
    
    if row[period] == 'MNTH':
        delta = relativedelta(months=row[freq])
    elif row[period] == 'YEAR':
        delta = relativedelta(years=row[freq])
    elif row[period] == 'DAIL':
        delta = timedelta(days=row[freq])
    else:
        raise ValueError(f"Unknown frequency period: {row[period]}")
    
    while start_date <= end_date:
        cashflows.append(start_date)
        start_date += delta

    return cashflows

def convert_to_years(value):
    
    if "M" in value:
        return int(str(value)[:-1])/12
    elif "Y" in value:
        return int(str(value)[:-1])
    else:
        return 0

def floating_rate(years_to_payout, float_rates_df, bump):
    differences = float_rates_df['Tenor'].sub(years_to_payout).abs()
    nearest_index = differences.idxmin()
    return (float_rates_df.loc[nearest_index, 'Rate'] + bump)/ 100

def calculate_discounted_cashflow(row, rate_type, bump, float_rates_df_list):
    total_discounted_cashflow = 0
    
    # CHECK
    discounted_cashflow_list = []
    cashflow_list = []
    factors = []
    pr_list = []
    discount_rate = []

    ibor = row['leg2UnderlierID']
    for item in float_rates_df_list:
        if item[0] == ibor:
            float_rates_df = item[1]
            break 
    
    for date in row['cashflow_dates']:

        if len(cashflow_list) == 0:

            time_difference = (date - datetime.now()).days / 365.0
            tenor = time_difference ### MONTHS IS TO MATCH THE INDEX IN OUR IBOR FILE
            float_rate = floating_rate(tenor,float_rates_df, bump)
            factor = (1 + float_rate)**time_difference
            factors.append(factor)
            discount_rate.append([tenor,1/ factor])

            if rate_type == 'float':
                payout_rate = float_rate
                cashflow = row['leg1NotionalAmount'] * payout_rate
                
            if rate_type == 'fixed':
                payout_rate = row['leg1FixedRate']
                cashflow = row['leg1NotionalAmount'] * payout_rate
                
            discounted_cashflow = cashflow / factor
        
        else:
            time_difference = ((date-row['cashflow_dates'][len(factors)-1])).days/365
            tenor = (date - datetime.now()).days / 365
            float_rate = floating_rate(tenor,float_rates_df,bump)
            prev_factor = factors[-1]
            factor = prev_factor * (1 + payout_rate) ** time_difference
            factors.append(factor)
            discount_rate.append([tenor,1/ factor])
            if rate_type == 'float':    
                payout_rate = float_rate
                cashflow = row['leg1NotionalAmount'] * payout_rate
            
            if rate_type == 'fixed':
                payout_rate = row['leg1FixedRate']
                cashflow = row['leg1NotionalAmount'] * row['leg1FixedRate']
                
            discounted_cashflow = cashflow / factor
        pr_list.append(payout_rate)
        total_discounted_cashflow += discounted_cashflow
        
        # CHECK
        cashflow_list.append(cashflow)
        discounted_cashflow_list.append(discounted_cashflow)
    
    # CHECK 
    #print('Total discounted CF '+rate_type+': '+str(total_discounted_cashflow))
    #print(rate_type + ' cashflow list ' + str((cashflow_list)))
    print(rate_type + ' '+ibor+' discount rates ' + str((discount_rate)))
    #print('\n')
    
    #print(rate_type + ' discounted cashflow list ' + str(discounted_cashflow_list) )
    #print(rate_type + ' discount factors ' + str((factors)))
    #print('\n')
    return total_discounted_cashflow

def fx_rates (your_path):
    exchange_rates_df = pd.read_csv(your_path + '7.IBOR/exchange_rates.csv')
    exchange_rates = dict(zip(exchange_rates_df['Currency'], exchange_rates_df['Rate_to_USD']))
    return exchange_rates

def fl_df (your_path, ibor):
    ibor_df = pd.read_csv(your_path + '7.IBOR/' + ibor + '.csv')
    ibor_df['Tenor'] = ibor_df['Tenor'].apply(convert_to_years) ## MAKE SURE ALL FILES ARE CODED THE SAME
    float_rates_df = ibor_df[['Tenor', 'Rate']].copy()
    return float_rates_df

def main (folder_path, your_path, bump):
    # 4. Data Loading
    # Load trade data
    files = [f for f in os.listdir(folder_path) if f != '.DS_Store']
    data_file = files[0][:-4]
    data = pd.read_csv(your_path + "2.Cleaned/" + data_file +'_Cleaned.csv')

    # Load exchange rates
    fx_df = fx_rates(your_path)
   
    # 5. Data Transformation and Filtering
    filtered_data = data[
        (data['leg1NotionalCurrency'].isin(['EUR', 'USD', 'GBP'])) & 
        (data['leg1UnderlyingAssetOrContractType'].isin(['Fixed-Floating', 'OIS']))
    ]
    df = pd.DataFrame(filtered_data)
    df['effectiveDate'] = pd.to_datetime(df['effectiveDate'])
    df['expirationDate'] = pd.to_datetime(df['expirationDate'])

    df['cashflow_dates'] = df.apply(
        lambda row: 
            calculate_cashflows(row, 'leg1FixedRatePaymentFrequencyPeriod', 'leg1FixedRatePaymentFrequencyMultiplier') 
            if row['leg1UnderlyingAssetOrContractType'] == 'Fixed-Floating' 
            else (calculate_cashflows(row, 'leg2UnderlierTenorPeriod', 'leg2UnderlierTenorMultiplier') 
                if row['leg1UnderlyingAssetOrContractType'] == 'OIS' 
                else None), 
        axis=1)

    # 6. Based on data - Load relevant IBORs to be used as float_rates in operations
    float_rates_df_list = []
    for ibor in df['leg2UnderlierID'].unique():
        float_rates_df_list.append([ibor,fl_df(your_path,ibor)])
        
    # 7. Main Operations
    df['leg1NotionalAmountUSD'] = df.apply(lambda row: convert_to_usd(row, fx_df), axis=1)
    df['MtM_leg1'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'fixed', 0, float_rates_df_list), axis=1)
    df['MtM_leg2'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'float', 0, float_rates_df_list), axis=1)
    df['MtM_leg2_bumped'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'float', bump/100, float_rates_df_list), axis=1)

    df['total_delta'] = (df['MtM_leg2'] - df['MtM_leg2_bumped']).abs()
    
    # 8. Export/Output
    df.drop(columns=['cashflow_dates', 'MtM_leg2_bumped'], inplace=True)
    df.to_csv(your_path + '3.Cash_Risk/' + data_file + '_Cash_Risk.csv', index=False)

# folder path, your path, bump (basis points, used to measure risk)
main(train_folder_path, your_path,bump=1)
main(test_folder_path, your_path,bump=1)


/var/folders/rq/30rpw85n53d2zxnt_c5qnrx40000gn/T/ipykernel_6692/1243126177.py:153: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['effectiveDate'] = pd.to_datetime(df['effectiveDate'])
/var/folders/rq/30rpw85n53d2zxnt_c5qnrx40000gn/T/ipykernel_6692/1243126177.py:154: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['expirationDate'] = pd.to_datetime(df['expirationDate'])


fixed USD-LIBOR-BBA discount rates [[-0.8328767123287671, 1.0461608693862567], [-0.33424657534246577, 1.032104557677014], [0.16712328767123288, 1.0181614300596171], [0.6684931506849315, 1.004406665923719], [1.16986301369863, 0.9908377205890918], [1.6684931506849314, 0.9775247356921835], [2.16986301369863, 0.9643189494784347], [2.6684931506849314, 0.9513622731797404], [3.16986301369863, 0.938509926499693], [3.6684931506849314, 0.9259000225593581], [4.16986301369863, 0.9133916559607715], [4.671232876712328, 0.9010522700632896], [5.1726027397260275, 0.8888795820366862], [5.671232876712328, 0.8769365158766805], [6.1726027397260275, 0.8650896175539523], [6.671232876712328, 0.853466195500419], [7.1726027397260275, 0.8419363674491019], [7.671232876712328, 0.8306240345502861], [8.172602739726027, 0.8194027907047219], [8.673972602739726, 0.8083331392863018], [9.175342465753424, 0.7974130323701891]]
fixed USD-LIBOR-BBA discount rates [[-0.8328767123287671, 1.0461608693862567], [-0.33424657534246

/var/folders/rq/30rpw85n53d2zxnt_c5qnrx40000gn/T/ipykernel_6692/1243126177.py:153: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['effectiveDate'] = pd.to_datetime(df['effectiveDate'])
/var/folders/rq/30rpw85n53d2zxnt_c5qnrx40000gn/T/ipykernel_6692/1243126177.py:154: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['expirationDate'] = pd.to_datetime(df['expirationDate'])


fixed EUR-EURIBOR discount rates [[-0.5835616438356165, 1.023611997600126], [0.4191780821917808, 1.0008773098707846], [1.4191780821917808, 0.9787076247548381], [2.419178082191781, 0.9570290037616298], [3.419178082191781, 0.9358305696968567], [4.421917808219178, 0.9150455302292545], [5.421917808219178, 0.8947770407032447], [6.421917808219178, 0.8749575033376404], [7.421917808219178, 0.8555769737287372], [8.424657534246576, 0.836574387424804], [9.424657534246576, 0.8180440535243702], [10.424657534246576, 0.7999241711983844], [11.424657534246576, 0.7822056488408418], [12.427397260273972, 0.7648326586763883], [13.427397260273972, 0.7478914221811411], [14.427397260273972, 0.7313254383516008], [15.427397260273972, 0.7151263952464777], [16.43013698630137, 0.6992432526875239], [17.43013698630137, 0.6837548380949962], [18.43013698630137, 0.6686094957962202], [19.43013698630137, 0.6537996266532703], [20.432876712328767, 0.6392785675172199], [21.432876712328767, 0.6251183858411463], [22.432876712

Now the same for the Test data